In [1]:
import gradio as gr
import networkx as nx
import matplotlib.pyplot as plt
from math import gcd
from itertools import product

# Constants
MAX_K_VALUE = 5
PRIMARY_COLOR = "#800000"
ACCENT_COLOR = "#3498db"
FRAME_BG = "#ecf0f1"

# Prime number checker
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

# Modulo group generator
def modulo_group(p, k):
    mod = p**k
    return [x for x in range(1, mod) if gcd(x, mod) == 1]

# Power relation checker
def is_power(u, v, mod):
    if u == 1 and v != 1:
        return False
    x = u
    while x != 1:
        x = (x * u) % mod
        if x == v:
            return True
    return v == 1 and u == 1

# Power graph generator
def power_graph(p, k):
    mod = p**k
    elements = modulo_group(p, k)
    G = nx.Graph()
    G.add_nodes_from(elements)

    for u, v in product(elements, repeat=2):
        if u != v and (is_power(u, v, mod) or is_power(v, u, mod)):
            G.add_edge(u, v)
    return G

# Graph indices calculations
def first_zagreb_index(G):
    return sum(d**2 for _, d in G.degree()) if G else 0

def wiener_index(G):
    if not G or not G.edges():
        return 0
    wiener_sum = 0
    nodes = list(G.nodes())
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            try:
                d = nx.shortest_path_length(G, source=nodes[i], target=nodes[j])
                wiener_sum += d
            except nx.NetworkXNoPath:
                continue
    return wiener_sum

def gutman_index(G):
    if not G or not G.edges():
        return 0
    gutman_sum = 0
    for i, u in enumerate(G.nodes()):
        for j, v in enumerate(G.nodes()):
            if i < j:
                try:
                    d = nx.shortest_path_length(G, source=u, target=v)
                    gutman_sum += d * G.degree[u] * G.degree[v]
                except nx.NetworkXNoPath:
                    continue
    return gutman_sum

def compute_indices(p, k_val, selected_indices):
    try:
        if p == 2 and k_val == 1:
            return {"zagreb": 0, "wiener": 0, "gutman": 0}

        G = power_graph(p, k_val)
        results = {}
        if "zagreb" in selected_indices:
            results["zagreb"] = first_zagreb_index(G)
        if "wiener" in selected_indices:
            results["wiener"] = wiener_index(G)
        if "gutman" in selected_indices:
            results["gutman"] = gutman_index(G)
        return results
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}

# Visualization functions
def show_graph(p, k):
    try:
        p = int(p)
        k = int(k)

        if not is_prime(p):
            return None, "Error: Prime must be a prime number ≥ 2"
        if k < 1 or k > MAX_K_VALUE:
            return None, f"Error: Exponent (k) must be between 1 and {MAX_K_VALUE}!"

        if p == 2 and k == 1:
            return None, "Graph cannot be visualized for p=2 and k=1 because the modulo group Z_(2^1)* is empty"

        G = power_graph(p, k)

        if G and len(G.nodes()) > 0:
            fig, ax = plt.subplots(figsize=(5, 4))
            pos = nx.circular_layout(G)
            nx.draw(G, pos, with_labels=True, node_color=ACCENT_COLOR,
                   edge_color=PRIMARY_COLOR, node_size=400,
                   font_size=8, ax=ax, width=1.5)
            ax.set_title(f"Power Graph of Z_({p}^{k})*", fontsize=9)
            ax.set_facecolor(FRAME_BG)
            fig.patch.set_facecolor(FRAME_BG)
            plt.close()
            return fig, None
        else:
            return None, "Graph cannot be created for this k value because the modulo group is empty."
    except Exception as e:
        return None, f"An error occurred while visualizing graph: {str(e)}"

def plot_indices(p, k_max, zagreb, wiener, gutman):
    try:
        p = int(p)
        k_max = int(k_max)

        if not is_prime(p):
            return None, "Error: Prime must be a prime number ≥ 2"
        if k_max < 1 or k_max > MAX_K_VALUE:
            return None, f"Error: Maximum exponent (k) must be between 1 and {MAX_K_VALUE}!"

        selected_indices = []
        if zagreb:
            selected_indices.append("zagreb")
        if wiener:
            selected_indices.append("wiener")
        if gutman:
            selected_indices.append("gutman")

        if not selected_indices:
            return None, "Error: Select at least one index to plot!"

        index_values = {index: [] for index in selected_indices}
        n_values = list(range(1, k_max + 1))

        for i in n_values:
            results = compute_indices(p, i, selected_indices)
            if results:
                for index in selected_indices:
                    index_values[index].append(results[index])

        if n_values and any(index_values.values()):
            fig, ax = plt.subplots(figsize=(6, 3.5))

            index_styles = {
                "zagreb": {"color": "#e74c3c", "marker": "o", "label": "First Zagreb Index"},
                "wiener": {"color": "#2ecc71", "marker": "s", "label": "Wiener Index"},
                "gutman": {"color": "#f39c12", "marker": "^", "label": "Gutman Index"}
            }

            for index in selected_indices:
                if index_values[index]:
                    style = index_styles[index]
                    ax.plot(n_values, index_values[index],
                           marker=style["marker"], linestyle='-',
                           color=style["color"],
                           label=style["label"])

            ax.set_xlabel("k (Prime Exponent)", fontsize=8)
            ax.set_ylabel("Graph Index Value", fontsize=8)
            ax.set_title(f"Index Changes for Power Graph Z_({p}^k)*", fontsize=9)
            ax.legend(fontsize=8)
            ax.grid(True, linestyle='--', alpha=0.5)
            ax.set_facecolor(FRAME_BG)
            fig.patch.set_facecolor(FRAME_BG)
            plt.close()
            return fig, None
        else:
            return None, "Not enough data to display index changes graph."
    except Exception as e:
        return None, f"An error occurred while creating plot: {str(e)}"

def calculate_indices(p, k, zagreb, wiener, gutman):
    try:
        p = int(p)
        k = int(k)

        if not is_prime(p):
            return "Error: Prime must be a prime number ≥ 2"
        if k < 1 or k > MAX_K_VALUE:
            return f"Error: Exponent (k) must be between 1 and {MAX_K_VALUE}!"

        selected_indices = []
        if zagreb:
            selected_indices.append("zagreb")
        if wiener:
            selected_indices.append("wiener")
        if gutman:
            selected_indices.append("gutman")

        if not selected_indices:
            return "Warning: Select at least one index to calculate!"

        results = compute_indices(p, k, selected_indices)

        if results:
            result_text = f"Results for p={p}, k={k}:\n\n"

            if p == 2 and k == 1:
                result_text += "• Special case: p=2 and k=1\n"
                result_text += "• All indices are 0 (modulo group Z_(2^1)* is empty)\n\n"
            else:
                if "zagreb" in results:
                    result_text += f"• First Zagreb Index: {results['zagreb']}\n"
                if "wiener" in results:
                    result_text += f"• Wiener Index: {results['wiener']}\n"
                if "gutman" in results:
                    result_text += f"• Gutman Index: {results['gutman']}\n"

            if k > 1:
                result_text += f"\nHistory from k=1 to k={k}:\n"
                for current_k in range(1, k + 1):
                    range_results = compute_indices(p, current_k, selected_indices)
                    if range_results:
                        if p == 2 and current_k == 1:
                            result_text += f"\nk=1 (p=2):\n"
                            result_text += "  • All indices = 0\n"
                            continue

                        result_text += f"\nk={current_k}:\n"
                        if "zagreb" in range_results:
                            result_text += f"  • Zagreb: {range_results['zagreb']}\n"
                        if "wiener" in range_results:
                            result_text += f"  • Wiener: {range_results['wiener']}\n"
                        if "gutman" in range_results:
                            result_text += f"  • Gutman: {range_results['gutman']}\n"

            return result_text
        else:
            return "Failed to calculate indices."
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Custom CSS for compact layout
css = """
.gradio-container {max-width: 1200px !important; height: 100vh !important}
.contain {display: flex !important; flex-direction: column !important; height: 100% !important}
#component-0 {height: 100% !important}
#tabs {height: calc(100% - 150px) !important; min-height: 500px !important}
.tab-item {height: 100% !important; overflow-y: auto !important}
#results, #graph, #plot {height: 100% !important}
#results textarea, #graph .plot-container, #plot .plot-container {height: 100% !important}
.compact {margin-bottom: 5px !important}
.small-plot .plot-container {height: 350px !important}
"""

# Create the Gradio interface
with gr.Blocks(
    title="Power Graph Analyzer",
    theme=gr.themes.Soft(primary_hue="red"),
    css=css
) as app:

    with gr.Row():
        # Left column - Inputs
        with gr.Column(scale=1, min_width=300):
            gr.Markdown("### Input Parameters", elem_classes="compact")

            with gr.Row():
                prime_input = gr.Number(
                    label="Prime (p)",
                    value=2,
                    precision=0,
                    min_width=100,
                    elem_classes="compact"
                )
                exponent_input = gr.Number(
                    label=f"Exponent (k: 1-{MAX_K_VALUE})",
                    value=1,
                    precision=0,
                    min_width=100,
                    elem_classes="compact"
                )

            gr.Markdown("**Select Indices:**", elem_classes="compact")
            with gr.Row():
                zagreb_check = gr.Checkbox(label="Zagreb", value=True)
                wiener_check = gr.Checkbox(label="Wiener", value=True)
                gutman_check = gr.Checkbox(label="Gutman", value=True)

            with gr.Row():
                calculate_btn = gr.Button("Calculate Index", variant="primary", size="sm")
                graph_btn = gr.Button("Show Graph", variant="primary", size="sm")
                plot_btn = gr.Button("Plot Graph", variant="primary", size="sm")

            gr.Markdown("""
            **Instructions:**
            1. Enter prime (p ≥ 2)
            2. Enter exponent (1 ≤ k ≤ 5)
            3. Select indices
            4. Click buttons to view results
            """, elem_classes="compact")

        # Right column - Outputs
        with gr.Column(scale=2):
            with gr.Tabs(elem_id="tabs"):
                with gr.Tab("Results", id="results"):
                    results_output = gr.Textbox(
                        lines=12,
                        max_lines=20,
                        show_copy_button=True,
                        container=False
                    )

                with gr.Tab("Graph Visualization", id="graph"):
                    graph_output = gr.Plot(elem_classes="small-plot")
                    graph_error = gr.Textbox(visible=False)

                with gr.Tab("Analysis Index", id="plot"):
                    plot_output = gr.Plot(elem_classes="small-plot")
                    plot_error = gr.Textbox(visible=False)

    # Button actions
    calculate_btn.click(
        fn=calculate_indices,
        inputs=[prime_input, exponent_input, zagreb_check, wiener_check, gutman_check],
        outputs=results_output
    )

    graph_btn.click(
        fn=show_graph,
        inputs=[prime_input, exponent_input],
        outputs=[graph_output, graph_error]
    )

    plot_btn.click(
        fn=plot_indices,
        inputs=[prime_input, exponent_input, zagreb_check, wiener_check, gutman_check],
        outputs=[plot_output, plot_error]
    )

# Launch the app
if __name__ == "__main__":
    app.launch(share=True, height=600)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7ce2537e22587bda84.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
